## Importing 'DDW-C18-0000.XLSX'(Language Data)  and 'DDW_PCA0000_2011_Indiastatedist.xlsx'(Census Data) file using pandas

In [1]:
import pandas as pd
data = pd.read_excel('INPUT/DDW-C18-0000.XLSX')
census_data = pd.read_excel('INPUT/DDW_PCA0000_2011_Indiastatedist.xlsx')

### Getting the index of rows containing state data for getting the total population of each state from census data

In [2]:
index = []
for i in range(len(census_data)):
    if census_data['Level'][i] == 'STATE':
        index.append(i)
for i in range(0,len(index),3):
    index[int(i/3)] = index[i]
index = index[:35]
index.insert(0,0)
index

[0,
 3,
 72,
 111,
 174,
 180,
 222,
 288,
 318,
 420,
 636,
 753,
 768,
 819,
 855,
 885,
 912,
 927,
 951,
 1035,
 1095,
 1170,
 1263,
 1320,
 1473,
 1554,
 1563,
 1569,
 1677,
 1749,
 1842,
 1851,
 1857,
 1902,
 2001,
 2016]

### Getting the names of states from census data and language data and checking if they both have the same states with same sequence

In [3]:
state_census = [0]*(len(index))
for i in range(len(index)):
    state_census[i] = census_data['Name'][index[i]]
state_census[0] = 'INDIA'
state_lang = data['Unnamed: 2'][6:].unique()
yes = 0
for i in range(len(state_census)):
    if state_census[i] == state_lang[i]:
        yes = yes+1
print(yes)

36


### Since the states list from both census and language data are same we can directly corresponding the states list of both data
### Getting the total population for each state from census data for males and females seperately

In [4]:
state_pop_male = [0]*len(index)
state_pop_female = [0]*len(index)
for i in range(len(index)):
    state_pop_male[i] = census_data['TOT_M'][index[i]]
    state_pop_female[i] = census_data['TOT_F'][index[i]]

### For Males and Females seperately:
### Getting the 'Number speaking second language' and 'Number speaking third language' from language data
### People speaking 3 or more languages is given directly in the data

In [5]:
state_2lang_male = [0] * len(index)
state_2lang_female = [0] * len(index)
state_threelang_male = [0] * len(index)
state_threelang_female = [0] * len(index)
for i in range(len(index)):
    state_2lang_male[i] = data['Unnamed: 6'][list(data['Unnamed: 2']).index(state_census[i])]
    state_2lang_female[i] = data['Unnamed: 7'][list(data['Unnamed: 2']).index(state_census[i])]
    state_threelang_male[i] = data['Unnamed: 9'][list(data['Unnamed: 2']).index(state_census[i])]
    state_threelang_female[i] = data['Unnamed: 10'][list(data['Unnamed: 2']).index(state_census[i])]

### For Males and Females seperately:
### Getting the counts of people speaking only one language, exactly two languages and three languages or more for each state.
### People speaking only 1 language = Total Population - People speaking 2 or more language
### People speaking exactly 2 languages = People speaking 2 or more language - People speaking 3 or more language

In [6]:
state_onelang_male = [0] * len(index)
state_onelang_female = [0] * len(index)
state_twolang_male = [0] * len(index)
state_twolang_female = [0] * len(index)
for i in range(len(index)):
    state_onelang_male[i] = state_pop_male[i] - state_2lang_male[i]
    state_onelang_female[i] = state_pop_female[i] - state_2lang_female[i]
    state_twolang_male[i] = state_2lang_male[i] - state_threelang_male[i]
    state_twolang_female[i] = state_2lang_female[i] - state_threelang_female[i]

### Getting the percentage of Male and Female population speaking only one language, exactly two languages and three languages

In [7]:
state_onelang_percent_male = [0]*len(index)
state_onelang_percent_female = [0]*len(index)
state_twolang_percent_male = [0]*len(index)
state_twolang_percent_female = [0]*len(index)
state_threelang_percent_male = [0]*len(index)
state_threelang_percent_female = [0]*len(index)
for i in range(len(index)):
    state_onelang_percent_male[i] = (state_onelang_male[i] * 100)/state_pop_male[i]
    state_onelang_percent_female[i] = (state_onelang_female[i] * 100)/state_pop_female[i]
    state_twolang_percent_male[i] = (state_twolang_male[i] * 100)/state_pop_male[i]
    state_twolang_percent_female[i] = (state_twolang_female[i] * 100)/state_pop_female[i]
    state_threelang_percent_male[i] = (state_threelang_male[i] * 100)/state_pop_male[i]
    state_threelang_percent_female[i] = (state_threelang_female[i] * 100)/state_pop_female[i]

### Getting the codes of the states

In [8]:
state_code = [0] * len(index)
for i in range(len(index)):
    state_code[i] = data['C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX'][list(data['Unnamed: 2']).index(state_census[i])]

### Importing ttest_1samp from scipy.stats module to do statistical test and get P-VALUE

In [9]:
from scipy.stats import ttest_1samp

### Getting P-VALUE

In [10]:
ratio_pop = [0]*len(state_pop_male)
ratio_1 = [0]*len(state_pop_male)
ratio_2 = [0]*len(state_pop_male)
ratio_3 = [0]*len(state_pop_male)
p_val = [0]*len(state_pop_male)
for i in range(len(state_pop_male)):
    ratio_pop[i] = state_pop_male[i] / state_pop_female[i]
    ratio_1[i] = state_onelang_male[i] / state_onelang_female[i]
    ratio_2[i] = state_twolang_male[i] / state_twolang_female[i]
    ratio_3[i] = state_threelang_male[i] / state_threelang_female[i]
    p_val[i] = ttest_1samp([ratio_1[i],ratio_2[i],ratio_3[i]], ratio_pop[i], axis = 0)[1]

### Exporting state codes, male percentage, female percentage and p value to dataframes for one language, two languages and three languages

In [11]:
data_final = {'state-code':state_code,'male-percentage':state_onelang_percent_male,'female-percentage':state_onelang_percent_female,'p-value':p_val}
gender_india_a = pd.DataFrame(data_final)
data_final = {'state-code':state_code,'male-percentage':state_twolang_percent_male,'female-percentage':state_twolang_percent_female,'p-value':p_val}
gender_india_b = pd.DataFrame(data_final)
data_final = {'state-code':state_code,'male-percentage':state_threelang_percent_male,'female-percentage':state_threelang_percent_female,'p-value':p_val}
gender_india_c = pd.DataFrame(data_final)

### Exporting dataframes to 'gender-india-{part}.csv'

In [12]:
gender_india_a.to_csv('OUTPUT/gender-india-a.csv',index=False)
gender_india_b.to_csv('OUTPUT/gender-india-b.csv',index=False)
gender_india_c.to_csv('OUTPUT/gender-india-c.csv',index=False)